# Import Event Log

In [1]:
import pandas as pd
import numpy as np
import pm4py
from pm4py.objects.conversion.log import converter as log_converter

if __name__ == "__main__":
    # Read the XES file
    dataframe_log = pm4py.read_xes('../../data/logs/PrepaidTravelCost.xes')

    # If 'log' is already a DataFrame, add the @@case_index column directly
    case_indices = {case_id: idx for idx, case_id in enumerate(dataframe_log['case:concept:name'].unique())}
    dataframe_log['@@case_index'] = dataframe_log['case:concept:name'].map(case_indices)
    
     # Convert the dataframe to event log
    log = log_converter.apply(dataframe_log)
    
dataframe_log

/opt/anaconda3/lib/python3.9/site-packages/pm4py/util/dt_parsing/parser.py:76: UserWarning: ISO8601 strings are not fully supported with strpfromiso for Python versions below 3.11
  warnings.warn(


parsing log, completed traces ::   0%|          | 0/2099 [00:00<?, ?it/s]

,id,org:resource,concept:name,time:timestamp,org:role,case:Rfp_id,case:Permit travel permit number,case:Task,case:OrganizationalEntity,case:RequestedAmount,...,case:Permit ProjectNumber,case:Project,case:concept:name,case:Permit OrganizationalEntity,case:Permit RequestedBudget,case:Cost Type,case:Permit id,case:Permit ActivityNumber,case:RfpNumber,@@case_index
0,st_step 73555_0,STAFF MEMBER,Permit SUBMITTED by EMPLOYEE,2017-01-09 13:48:43+00:00,EMPLOYEE,request for payment 73550,UNKNOWN,task 71977,organizational unit 65463,854.579838,...,UNKNOWN,project 503,request for payment 73550,organizational unit 65455,1979.272104,0,travel permit 73549,UNKNOWN,request for payment number 73551,0
1,st_step 73554_0,STAFF MEMBER,Permit FINAL_APPROVED by SUPERVISOR,2017-01-09 13:48:55+00:00,SUPERVISOR,request for payment 73550,UNKNOWN,task 71977,organizational unit 65463,854.579838,...,UNKNOWN,project 503,request for payment 73550,organizational unit 65455,1979.272104,0,travel permit 73549,UNKNOWN,request for payment number 73551,0
2,st_step 73558_0,STAFF MEMBER,Request For Payment SUBMITTED by EMPLOYEE,2017-01-12 10:40:27+00:00,EMPLOYEE,request for payment 73550,UNKNOWN,task 71977,organizational unit 65463,854.579838,...,UNKNOWN,project 503,request for payment 73550,organizational unit 65455,1979.272104,0,travel permit 73549,UNKNOWN,request for payment number 73551,0
3,st_step 73559_0,STAFF MEMBER,Request For Payment FINAL_APPROVED by SUPERVISOR,2017-01-12 10:41:59+00:00,SUPERVISOR,request for payment 73550,UNKNOWN,task 71977,organizational unit 65463,854.579838,...,UNKNOWN,project 503,request for payment 73550,organizational unit 65455,1979.272104,0,travel permit 73549,UNKNOWN,request for payment number 73551,0
4,st_step 73557_0,STAFF MEMBER,Request For Payment REJECTED by MISSING,2017-01-12 10:53:07+00:00,MISSING,request for payment 73550,UNKNOWN,task 71977,organizational unit 65463,854.579838,...,UNKNOWN,project 503,request for payment 73550,organizational unit 65455,1979.272104,0,travel permit 73549,UNKNOWN,request for payment number 73551,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18241,st_step 186614_0,STAFF MEMBER,Request For Payment SUBMITTED by EMPLOYEE,2018-12-30 19:16:15+00:00,EMPLOYEE,request for payment 186612,UNKNOWN,UNKNOWN,organizational unit 65462,96.739075,...,UNKNOWN,project 147531,request for payment 186612,UNKNOWN,0.000000,0,UNKNOWN,UNKNOWN,request for payment number 186613,2098
18242,st_step 186616_0,STAFF MEMBER,Request For Payment APPROVED by ADMINISTRATION,2018-12-30 19:16:25+00:00,ADMINISTRATION,request for payment 186612,UNKNOWN,UNKNOWN,organizational unit 65462,96.739075,...,UNKNOWN,project 147531,request for payment 186612,UNKNOWN,0.000000,0,UNKNOWN,UNKNOWN,request for payment number 186613,2098
18243,st_step 186615_0,STAFF MEMBER,Request For Payment FINAL_APPROVED by SUPERVISOR,2019-01-14 14:09:11+00:00,SUPERVISOR,request for payment 186612,UNKNOWN,UNKNOWN,organizational unit 65462,96.739075,...,UNKNOWN,project 147531,request for payment 186612,UNKNOWN,0.000000,0,UNKNOWN,UNKNOWN,request for payment number 186613,2098
18244,rp_request for payment 186612_15,SYSTEM,Request Payment,2019-01-15 06:02:45+00:00,UNDEFINED,request for payment 186612,UNKNOWN,UNKNOWN,organizational unit 65462,96.739075,...,UNKNOWN,project 147531,request for payment 186612,UNKNOWN,0.000000,0,UNKNOWN,UNKNOWN,request for payment number 186613,2098


# Preprocess

In [2]:
codes, uniques = pd.factorize(dataframe_log['concept:name'])
dataframe_log['concept:name'] = codes + 1

# Generate Prefixes

In [3]:
df_activity = dataframe_log[['concept:name', '@@case_index']]

In [4]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

def generate_prefix_windows(df, case_id_column='@@case_index', max_len=None):
    windows = []
    targets = []
    case_indices = []
    
    for case_id in df[case_id_column].unique():
        case_data = df[df[case_id_column] == case_id].drop(columns=[case_id_column]).to_numpy()
        
        # Optional: Make sure to sort the case data if there's an implicit order (e.g., by timestamps)
        # case_data = case_data.sort_values(by='timestamp_column').to_numpy()  # Uncomment and adjust if needed
        
        for i in range(1, len(case_data)):
            window = case_data[:i]
            target = case_data[i]
            windows.append(window)
            targets.append(target)
            case_indices.append(case_id)
    
    if max_len is None:
        max_len = max(len(window) for window in windows)
    
    # Pad sequences
    windows_padded = pad_sequences(windows, maxlen=max_len, padding='post', dtype='float32')
    
    # Convert targets to numpy array
    targets_array = np.array(targets, dtype='float32')
    case_indices_array = np.array(case_indices)
    
    return windows_padded, targets_array, case_indices_array

2024-08-19 09:57:17.846382: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
windows_activity, targets_activity, case_indices = generate_prefix_windows(df_activity)

# GRU

### Architecture

- Separate Inputs for Each Attribute
- Each attribute is passed through an embedding layer
- Each attribute has its corresponding GRU encoder
- Selective Concatenation: After encoding, the outputs of these GRU layers are concatenated. However, this concatenation is selective, meaning it is structured in a way that prepares the data for effective synthesis without leaking information from the future (next event attributes)
- Decoder GRUs: Integrated Decoding: Post-concatenation, the combined attributes are processed through decoder GRU layers. These layers are tasked with integrating the data from different attributes and preparing it for final prediction. This step is where BINet v3 distinguishes itself by effectively using the interdependencies between different attributes to enhance prediction accuracy.
- Output Layer: Softmax Output for Each Attribute: For each attribute of the next event, a softmax layer predicts a probability distribution over all possible values. This allows the model to output the most likely next event and its attributes based on the learned dependencies and the history encoded by the GRUs.
- E: maximum case length
- We train BINet with a GRU size of 2E (two times the maximum case length)
- on mini batches of size 500 for 20 epochs

In [6]:
# Group by the @@case_index column and count the rows in each group
case_lengths = dataframe_log.groupby('@@case_index').size()

# Find the maximum value among the case lengths
E = case_lengths.max()

In [7]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU, Embedding, Dense, Dropout, BatchNormalization

def create_binetv3_for_activity(num_activities, embedding_dim, gru_units, dropout_rate):
    # Input layers for activity attribute
    input_activity = Input(shape=(None,), name='activity_input')

    # Embedding layer for activity attribute
    embedding_activity = Embedding(input_dim=num_activities + 1, output_dim=embedding_dim, mask_zero=True, name='activity_embedding')(input_activity)

    # Encoder GRU with Batch Normalization for activity
    encoded_activity = GRU(units=gru_units, return_sequences=False, name='activity_encoder')(embedding_activity)
    bn_activity = BatchNormalization(name='bn_activity')(encoded_activity)

    # Dropout layer
    dropout_layer = Dropout(rate=dropout_rate, name='dropout')(bn_activity)

    # Output layer for predicting the next activity
    output_activity = Dense(num_activities + 1, activation='softmax', name='output_activity')(dropout_layer)

    # Building the model
    model = Model(inputs=input_activity, outputs=output_activity)
    model.compile(
        optimizer='adam', 
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# Parameters
gru_units = int(2 * E) 
num_activities = dataframe_log['concept:name'].max()
embedding_dim = 50
dropout_rate = 0.2
model = create_binetv3_for_activity(num_activities, embedding_dim, gru_units, dropout_rate)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 activity_input (InputLayer  [(None, None)]            0         
 )                                                               
                                                                 
 activity_embedding (Embedd  (None, None, 50)          1500      
 ing)                                                            
                                                                 
 activity_encoder (GRU)      (None, 42)                11844     
                                                                 
 bn_activity (BatchNormaliz  (None, 42)                168       
 ation)                                                          
                                                                 
 dropout (Dropout)           (None, 42)                0         
                                                             

### Data Splitting

In [8]:
from sklearn.model_selection import train_test_split

train_activity, test_activity, train_targets_activity, test_targets_activity = train_test_split(
    windows_activity, targets_activity, test_size=0.3, random_state=42)


### Training

In [9]:
from tensorflow.keras.utils import to_categorical

train_targets_activity_cat = to_categorical(train_targets_activity, num_classes=num_activities + 1)
test_targets_activity_cat = to_categorical(test_targets_activity, num_classes=num_activities + 1)

In [10]:
history = model.fit(
    train_activity,  # Only the activity input
    train_targets_activity_cat,  # Only the activity targets
    validation_data=(test_activity, test_targets_activity_cat),  # Only the activity input and targets for validation
    epochs=20,
    batch_size=500
)

Epoch 1/20
23/23 [==============================] - 7s 94ms/step - loss: 2.2727 - accuracy: 0.5674 - val_loss: 3.2215 - val_accuracy: 0.5232
Epoch 2/20
23/23 [==============================] - 1s 34ms/step - loss: 1.1202 - accuracy: 0.7784 - val_loss: 3.0608 - val_accuracy: 0.5901
Epoch 3/20
23/23 [==============================] - 1s 33ms/step - loss: 0.8111 - accuracy: 0.7927 - val_loss: 2.9468 - val_accuracy: 0.7251
Epoch 4/20
23/23 [==============================] - 1s 33ms/step - loss: 0.7058 - accuracy: 0.8063 - val_loss: 2.8416 - val_accuracy: 0.7523
Epoch 5/20
23/23 [==============================] - 1s 36ms/step - loss: 0.6325 - accuracy: 0.8234 - val_loss: 2.7333 - val_accuracy: 0.7670
Epoch 6/20
23/23 [==============================] - 1s 40ms/step - loss: 0.5899 - accuracy: 0.8386 - val_loss: 2.6180 - val_accuracy: 0.7940
Epoch 7/20
23/23 [==============================] - 1s 33ms/step - loss: 0.5560 - accuracy: 0.8414 - val_loss: 2.4985 - val_accuracy: 0.8208
Epoch 8/20
23

In [11]:
# Evaluate the model on the validation set
results = model.evaluate(
    test_activity,  # Only the activity input
    test_targets_activity_cat,  # Only the activity targets
    batch_size=64
)
print(f"Validation Loss: {results[0]}, Validation Accuracy: {results[1]}")

76/76 [==============================] - 0s 3ms/step - loss: 0.6320 - accuracy: 0.8541
Validation Loss: 0.6320372223854065, Validation Accuracy: 0.8540763854980469


In [12]:
# Save the model to an H5 file
model.save('prepaid.h5')

/opt/anaconda3/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Anomaly Score Computation

- For each event attribute, BINet's softmax layer outputs a probability distribution over possible values
- The anomaly score for a specific attribute value v is calculated by summing all the probabilities from the softmax output that are greater than the probability assigned to v

In [13]:
# Generate predictions for all inputs
predictions_activity = model.predict([windows_activity])

505/505 [==============================] - 2s 2ms/step


In [14]:
import numpy as np

def calculate_anomaly_scores(predictions, targets):
    scores = []
    # Loop through each example in the predictions
    for i in range(predictions.shape[0]):
        actual_prob = predictions[i, targets[i]]  # Extract the probability of the true class using target index
        # Calculate anomaly score as sum of probabilities greater than the probability of the actual value
        anomaly_score = np.sum(predictions[i][predictions[i] > actual_prob])
        scores.append(anomaly_score)

    return scores

In [15]:
# Calculate anomaly scores for each attribute type
anomaly_scores_activity = calculate_anomaly_scores(predictions_activity, targets_activity.astype(int))

## Insert missing scores for cases with less than 2 Events

In [16]:
import pandas as pd

# Create a DataFrame from the case_indices_array corresponding to case_resource
score = pd.DataFrame({'case': case_indices})
score['score_activity'] = anomaly_scores_activity

score['case'] = score['case'].astype(int)

score

,case,score_activity
0,0,0.525272
1,0,0.000000
2,0,0.000000
3,0,0.835291
4,0,0.000000
...,...,...
16142,2097,0.000000
16143,2098,0.000000
16144,2098,0.493282
16145,2098,0.000000


In [17]:
import pandas as pd

def contains_all_values(df, column, end):

    # Generate the set of all values in the specified range
    required_values = set(range(0, end + 1))
    
    # Get the unique values in the specified column
    column_values = set(df[column].unique())
    
    # Find missing values
    missing_values = required_values - column_values
    
    # Print missing values if any
    if missing_values:
        print(f"Missing values: {sorted(missing_values)}")
    
    # Check if all required values are in the column values
    return required_values.issubset(column_values)

end = 2098

result = contains_all_values(score, 'case', end)
print(f"Does the 'case' column contain all values between 0 and {end}? {result}")



Missing values: [87, 166, 1081, 1139, 1301, 1586, 1835]
Does the 'case' column contain all values between 0 and 2098? False


In [18]:
# Generate the full range of values
full_range = set(range(2098+1))

# Get the existing 'case' values
existing_cases = set(score['case'])

# Identify missing values
missing_cases = full_range - existing_cases

# Create a DataFrame for missing values
missing_df = pd.DataFrame({'case': list(missing_cases)})

# Add other columns with value 1
for col in score.columns:
    if col != 'case':
        missing_df[col] = 1

# Concatenate the original DataFrame with the missing values DataFrame
updated_score = pd.concat([score, missing_df])

# Sort the DataFrame by 'case'
updated_score = updated_score.sort_values(by='case').reset_index(drop=True)

updated_score 

,case,score_activity
0,0,0.525272
1,0,0.000000
2,0,0.000000
3,0,0.835291
4,0,0.000000
...,...,...
16149,2097,0.000000
16150,2098,0.493282
16151,2098,0.000000
16152,2098,0.000000


In [19]:
anomaly_scores_activity = updated_score['score_activity']

### Threshold (lowest plateau)

In [20]:
import numpy as np

def calculate_anomaly_ratio(scores, threshold):
    """
    Calculate the anomaly ratio for a given threshold.
    """
    return np.mean(scores > threshold)

def find_plateaus(scores, epsilon=1e-4, min_plateau_length=10):
    """
    Identify the lowest plateau in the anomaly ratio function and calculate the mean-centered threshold.
    """
    scores = np.array(scores)  # Convert scores to a NumPy array
    sorted_scores = np.sort(scores)
    
    # Remove duplicate values
    unique_thresholds, unique_indices = np.unique(sorted_scores, return_index=True)
    anomaly_ratios = np.array([calculate_anomaly_ratio(scores, t) for t in unique_thresholds])
    
    # Calculate first and second derivatives
    first_derivatives = np.diff(anomaly_ratios) / np.diff(unique_thresholds)
    second_derivatives = np.diff(first_derivatives) / np.diff(unique_thresholds[:-1])
    
    # Identify plateaus where the first derivative is close to zero
    plateau_indices = np.where(np.abs(first_derivatives) < epsilon)[0]
    
    # Group consecutive indices to identify continuous plateaus
    grouped_plateaus = np.split(plateau_indices, np.where(np.diff(plateau_indices) != 1)[0] + 1)
    
    # Filter plateaus based on minimum length
    long_plateaus = [g for g in grouped_plateaus if len(g) >= min_plateau_length]
    
    if long_plateaus:
        # Take the first long plateau and find the mean threshold in this plateau
        first_plateau = long_plateaus[0]
        plateau_thresholds = unique_thresholds[first_plateau]
        return np.mean(plateau_thresholds)
    else:
        # If no plateau is found, return a default value, e.g., the 90th percentile
        percentile_90 = np.percentile(sorted_scores, 90)
        if percentile_90 == 1.0:
            return 0.4
        else:
            return percentile_90


In [21]:
threshold_activity = find_plateaus(anomaly_scores_activity)

### Detection

In [22]:
def detect_anomalies(anomaly_scores, threshold):
    labels = [1 if score > threshold else 0 for score in anomaly_scores]
    return labels

In [23]:
# Detect anomalies based on the calculated anomaly scores and thresholds
labels_activity = detect_anomalies(anomaly_scores_activity, threshold_activity)

# Mapping

In [24]:
case_indices = updated_score['case']

In [25]:
import pandas as pd

# Create a DataFrame from the case_indices_array corresponding to case_resource
mapping = pd.DataFrame({'case': case_indices})
mapping['predicted_activity'] = labels_activity

mapping

,case,predicted_activity
0,0,1
1,0,0
2,0,0
3,0,1
4,0,0
...,...,...
16149,2097,0
16150,2098,1
16151,2098,0
16152,2098,0


In [26]:
# Create a boolean DataFrame where each value is True if the value is 1
contains_one = (mapping[['predicted_activity']] == 1)

# Group by 'case' and check if there's at least one 'True' in any of the columns
case_prediction = contains_one.groupby(mapping['case']).any().any(axis=1)
case_prediction

case
0        True
1        True
2        True
3        True
4        True
        ...  
2094    False
2095     True
2096     True
2097    False
2098     True
Length: 2099, dtype: bool

# Ground Truth

In [27]:
def generate_alignments_adjusted_tracecost_pkl(log, net, initial_marking, final_marking):
    from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments
    from pm4py.algo.conformance.alignments.petri_net import variants
    from pm4py.objects.petri_net.utils import align_utils
    max_events=0
    for trace in log:
        counter=0
        for event in trace:
            counter+=1
        if counter > max_events:
            max_events=counter
    parameters={}
    parameters[alignments.Variants.VERSION_STATE_EQUATION_A_STAR.value.Parameters.PARAM_SYNC_COST_FUNCTION] = list(map(lambda i: .1*i, range(max_events*2)))
    parameters[alignments.Variants.VERSION_STATE_EQUATION_A_STAR.value.Parameters.PARAM_TRACE_COST_FUNCTION]=list(map(lambda i: align_utils.STD_MODEL_LOG_MOVE_COST-.1*i, range(max_events*2)))
    aligned_traces = alignments.apply_log(log, net, initial_marking, final_marking, variant=variants.state_equation_a_star, parameters=parameters)
    return aligned_traces

In [28]:
import pm4py
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.bpmn.importer import importer as bpmn_importer
from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments_petri

# 2. Import the given BPMN model
bpmn_graph = bpmn_importer.apply("../../data/model/Model_PrepaidTravelCost.bpmn")

# 3. Convert the BPMN to a Petri net
net, initial_marking, final_marking = pm4py.convert_to_petri_net(bpmn_graph)

aligned_traces = generate_alignments_adjusted_tracecost_pkl(log, net, initial_marking, final_marking)

aligning log, completed variants ::   0%|          | 0/202 [00:00<?, ?it/s]

In [29]:
def extract_conformance_status_by_fitness(aligned_traces):
    conformance_status = []
    for alignment in aligned_traces:
        fitness = alignment['fitness']
        # If the fitness is 1.0, the trace is conforming
        if fitness == 1.0:
            conformance_status.append(0)
        else:
            conformance_status.append(1)
    return conformance_status

# Get the conformance status list from the aligned traces
conformance = extract_conformance_status_by_fitness(aligned_traces)

In [30]:
ground_truth = pd.DataFrame({'conformity': conformance})
ground_truth['predicted'] = case_prediction

# Convert False to 0 and True to 1
ground_truth['predicted'] = [int(value) for value in ground_truth['predicted']]
ground_truth

,conformity,predicted
0,1,1
1,1,1
2,1,1
3,1,1
4,1,1
...,...,...
2094,1,0
2095,1,1
2096,1,1
2097,1,0


# Evaluation

In [31]:
# Calculating TP, TN, FP, FN
TP = ((ground_truth['conformity'] == 1) & (ground_truth['predicted'] == 1)).sum()
TN = ((ground_truth['conformity'] == 0) & (ground_truth['predicted'] == 0)).sum()
FP = ((ground_truth['conformity'] == 0) & (ground_truth['predicted'] == 1)).sum()
FN = ((ground_truth['conformity'] == 1) & (ground_truth['predicted'] == 0)).sum()

In [32]:
precision_dev = TP / (TP + FP)
print(f"Precision Dev: {precision_dev:.2f}")

Precision Dev: 0.72


In [33]:
recall_dev = TP / (TP + FN)
print(f"Recall Dev: {recall_dev:.2f}")

Recall Dev: 0.90


In [34]:
precision_no_dev = TN / (TN + FN)
print(f"Precision No Dev: {precision_no_dev:.2f}")

Precision No Dev: 0.90


In [35]:
recall_no_dev = TN / (TN + FP)
print(f"Recall No Dev: {recall_no_dev:.2f}")

Recall No Dev: 0.74


In [36]:
from sklearn.metrics import roc_auc_score

auc_roc = roc_auc_score(ground_truth['conformity'], ground_truth['predicted'])
print(f"AUC-ROC: {auc_roc:.2f}")

AUC-ROC: 0.82
